In [ ]:
# Import libraries
import os
import datetime
import pathlib
import requests
import io
import shutil
from glob import glob

# from qgis.core import *

import earthpy as et
import geopandas as gpd
import marxanconpy as mx
import numpy as np
import pandas as pd

import kba_thresh_sa_scripts as ks

# set global cache override variable
CACHE_OVERRIDE = False

In [ ]:
# open full hex.shp file
fullhex_data_path = os.path.join(data_path, "hex_shp", 'Hex_7mi2.shp')
fullhex_layer = gpd.read_file(fullhex_data_path)

# Reproject CRS to ESPG 5070
fullhex_layer_5070 = fullhex_layer.to_crs(epsg='5070')